# Prediction
We will use rolling window to average the stats of the player, and use them as the input for prediction.

In [172]:
import pandas as pd
import time
import os
from datetime import datetime
import numpy as np

In [173]:
df = pd.read_csv('source/summary.csv')

In [174]:
df = df.sort_values(by = ['Unnamed: 0_level_0_Player','Unnamed: 4_level_0_Age'])

In [175]:
df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_#,Unnamed: 2_level_0_Nation,Unnamed: 3_level_0_Pos,Unnamed: 4_level_0_Age,Unnamed: 5_level_0_Min,Performance_Gls,Performance.1_Ast,Performance.2_PK,Performance.3_PKatt,...,SCA.1_GCA,Passes_Cmp,Passes.1_Att,Passes.2_Cmp%,Passes.3_PrgP,Carries_Carries,Carries.1_PrgC,Take-Ons_Att,Take-Ons.1_Succ,Team_Team
51482,Aaron Connolly,44.0,ie IRL,FW,19-215,25,0,0,0,0,...,0,2,2,100.0,0,1,0,0,0,Brighton & Hove Albion
14649,Aaron Connolly,44.0,ie IRL,FW,19-229,7,0,0,0,0,...,0,1,1,100.0,0,2,1,0,0,Brighton & Hove Albion
60635,Aaron Connolly,44.0,ie IRL,"FW,LW",19-236,21,0,0,0,0,...,0,5,6,83.3,0,7,1,0,0,Brighton & Hove Albion
22904,Aaron Connolly,44.0,ie IRL,FW,19-243,26,0,0,0,0,...,0,3,3,100.0,0,3,0,1,1,Brighton & Hove Albion
17582,Aaron Connolly,44.0,ie IRL,FW,19-250,79,2,0,0,0,...,1,10,15,66.7,0,18,2,3,1,Brighton & Hove Albion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82294,Łukasz Fabiański,1.0,pl POL,GK,39-247,90,0,0,0,0,...,0,37,56,66.1,0,32,0,0,0,West Ham United
72328,Łukasz Fabiański,1.0,pl POL,GK,39-252,35,0,0,0,0,...,0,15,18,83.3,0,15,0,1,1,West Ham United
83450,Łukasz Fabiański,1.0,pl POL,GK,39-271,90,0,0,0,0,...,0,41,52,78.8,0,26,0,0,0,West Ham United
83038,Łukasz Fabiański,1.0,pl POL,GK,39-275,90,0,0,0,0,...,0,32,36,88.9,0,19,0,1,1,West Ham United


# Load Model

In [176]:
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

model = xgb.XGBClassifier()
model.load_model('model/' + "model.json")

# Make a prediction
Set that wanna predict Manchester United vs. Liverpool (based on current performance (summary.csv))

In [177]:
manutd_squad = [
    'Bruno Fernandes', 'Marcus Rashford', 'Casemiro', 'Lisandro Martínez',
    'Raphaël Varane', 'André Onana', 'Alejandro Garnacho', 'Kobbie Mainoo',
    'Antony', 'Mason Mount', 'Harry Maguire', 'Luke Shaw'
] # Need the user to select their own squad
liverpool_squad = [
    'Mohamed Salah', 'Virgil van Dijk', 'Alisson Becker', 'Trent Alexander-Arnold',
    'Darwin Núñez', 'Luis Díaz', 'Andy Robertson', 'Dominik Szoboszlai',
    'Alexis Mac Allister', 'Cody Gakpo', 'Diogo Jota', 'Ibrahima Konaté'
] # Need the user to select their own squad

In [178]:
df[df['Unnamed: 0_level_0_Player'] == manutd_squad[11]]

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_#,Unnamed: 2_level_0_Nation,Unnamed: 3_level_0_Pos,Unnamed: 4_level_0_Age,Unnamed: 5_level_0_Min,Performance_Gls,Performance.1_Ast,Performance.2_PK,Performance.3_PKatt,...,SCA.1_GCA,Passes_Cmp,Passes.1_Att,Passes.2_Cmp%,Passes.3_PrgP,Carries_Carries,Carries.1_PrgC,Take-Ons_Att,Take-Ons.1_Succ,Team_Team
55699,Luke Shaw,23.0,eng ENG,LB,22-154,82,0,0,0,0,...,0,44,60,73.3,2,34,2,1,1,Manchester United
56371,Luke Shaw,23.0,eng ENG,LB,22-167,90,0,0,0,0,...,0,57,83,68.7,7,47,6,1,0,Manchester United
58581,Luke Shaw,23.0,eng ENG,LB,22-171,90,0,0,0,0,...,0,54,69,78.3,8,52,8,2,1,Manchester United
33724,Luke Shaw,23.0,eng ENG,LB,22-173,90,0,0,0,0,...,0,49,64,76.6,6,31,0,0,0,Manchester United
58775,Luke Shaw,23.0,eng ENG,LB,22-187,90,0,0,0,0,...,0,58,69,84.1,7,44,2,0,0,Manchester United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29380,Luke Shaw,23.0,eng ENG,CB,30-141,81,0,0,0,0,...,0,60,69,87.0,6,51,3,0,0,Manchester United
59313,Luke Shaw,23.0,eng ENG,CB,30-145,87,0,0,0,0,...,0,53,74,71.6,6,44,0,1,0,Manchester United
88820,Luke Shaw,23.0,eng ENG,CB,30-149,90,0,0,0,0,...,0,42,50,84.0,1,31,1,0,0,Manchester United
55729,Luke Shaw,23.0,eng ENG,"CB,WB",30-156,89,0,0,0,0,...,0,33,43,76.7,1,24,2,1,1,Manchester United


In [179]:
home_df = []
away_df = []
for player in manutd_squad:
    player_df = df[df['Unnamed: 0_level_0_Player'] == player].iloc[-3:,:]
    player_df_average = player_df.groupby('Unnamed: 0_level_0_Player').mean(numeric_only = True)
    player_df_average['Team_Team'] = player_df['Team_Team'].unique()[-1]
    player_df_average['Unnamed: 3_level_0_Pos'] = player_df['Unnamed: 3_level_0_Pos'].unique()[-1]
    home_df.append(player_df_average)
home_df = pd.concat(home_df)
for player in manutd_squad:
    player_df = df[df['Unnamed: 0_level_0_Player'] == player].iloc[-3:,:]
    player_df_average = player_df.groupby('Unnamed: 0_level_0_Player').mean(numeric_only = True)
    player_df_average['Team_Team'] = player_df['Team_Team'].unique()[-1]
    player_df_average['Unnamed: 3_level_0_Pos'] = player_df['Unnamed: 3_level_0_Pos'].unique()[-1]
    away_df.append(player_df_average)
away_df = pd.concat(away_df)

In [180]:
home_df['Unnamed: 5_level_0_Min'] = 90.0
away_df['Unnamed: 5_level_0_Min'] = 90.0

In [181]:
def extract_team_features(team_df, is_home = True):
    features = {}


    features['avg_minutes'] = team_df['Unnamed: 5_level_0_Min'].astype('float32').mean()
    # Shooting
    features['total_shots'] = team_df['Performance.4_Sh'].astype('float32').sum()
    features['shots_on_target'] = team_df['Performance.5_SoT'].astype('float32').sum()
    features['xG'] = team_df['Expected_xG'].astype('float32').sum()
    features['xAG'] = team_df['Expected.2_xAG'].astype('float32').sum()
    # Passing
    features['key_passes'] = team_df['SCA_SCA'].astype('float32').sum() # Shot creating action
    features['pass_completion'] = team_df['Passes_Cmp'].astype('float32').sum()/team_df['Passes.1_Att'].astype('float32').sum()*100

    # Defensive
    features['tackles'] = team_df['Performance.9_Tkl'].astype('float32').sum()
    features['interception'] = team_df['Performance.10_Int'].astype('float32').sum()
    features['blocks'] = team_df['Performance.11_Blocks'].astype('float32').sum()

    # Cards
    features['yellow_cards'] = team_df['Performance.6_CrdY'].astype('float32').sum()
    features['red_cards'] = team_df['Performance.7_CrdR'].astype('float32').sum()

    # Position-specific
    positions = team_df['Unnamed: 3_level_0_Pos'].astype(str)

    # Attackers (FW, LW, RW, ST)
    attackers = team_df[positions.str.contains('FW|LW|RW|ST|AM')]

    if len(attackers) > 0:
        features['attackers_xG'] = attackers['Expected_xG'].astype('float32').sum()
        features['attackers_shots'] = attackers['Performance.4_Sh'].astype('float32').sum()

    midfielders = team_df[positions.str.contains('CM|DM|LM|RM|AM')]
    
    if len(midfielders) > 0:
        features['midfielders_passes'] = midfielders['Passes_Cmp'].astype('float32').sum()/midfielders['Passes.1_Att'].astype('float32').sum()*100

    defenders = team_df[positions.str.contains('CB|RB|LB|WB|DF')]

    if len(defenders) > 0:
        features['defenders_tackles'] = defenders['Performance.9_Tkl'].astype('float32').sum()
        features['defenders_blocks'] = defenders['Performance.11_Blocks'].astype('float32').sum()

    return features

def process_match_file(home_df, away_df):
    home_data = home_df
    away_data = away_df
    home_features = extract_team_features(home_data, is_home = True)
    away_features = extract_team_features(away_data, is_home = False)

    match_features = {}
    for key, value in home_features.items():
        match_features[f'home_{key}'] = value

    for key, value in away_features.items():
        match_features[f'away_{key}'] = value

    for key in home_features.keys():
        if key in away_features:
            match_features[f'diff_{key}'] = home_features[key] - away_features[key]
            match_features[f'ratio_{key}'] = home_features[key]/(away_features[key] + 0.00001) # avoid dividing by zero

    return match_features 

In [182]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [183]:
X = process_match_file(home_df, away_df)

In [184]:
import joblib
match_df = pd.DataFrame([X])
scaler = joblib.load('scaler/scaler.pkl')

In [185]:
match_df_scaled = scaler.transform(match_df)

In [186]:
match_result = model.predict(match_df_scaled)

In [187]:
match_result # 0 = Home Win, 1 = Draw, 2 = Away Win

array([1], dtype=int64)